# Different transformer-based models experiments

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# Position into correct google drive folder - this may vary!

In [ ]:
%cd /content/gdrive/MyDrive/fer-tar

# Install required modules

In [ ]:
!pip install -q pytorch_lightning
!pip install -q transformers

# Import required modules

In [ ]:
from importlib import reload
import src.modelss
import my_scripts.data
reload(src.modelss)
reload(my_scripts.data)

In [ ]:
from src.modelss import TransformerBasedClassifier
from src.tokenizers import TransformerTokenizer
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from my_scripts.data import ComVEDataset
from torch.utils.data import DataLoader

# Initialize hyperparameters

In [ ]:
BATCH_SIZE = 32
EPOCHS = 8
# NAMES = ['bert', 'albert', 'roberta', 'electra']
NAMES = ['electra']
# AUG_SIZES = [5000, 10000]
AUG_SIZES = [5000]
DROP = 0.1

# Position into some subfolder because of Pytorch Lightning and its logging system...

In [ ]:
%cd Test

# Training loop and testing on evaluation set

In [ ]:
for name in NAMES:
    tokenizer = TransformerTokenizer(name=name)
    for size in AUG_SIZES:
        train_ds, val_ds, test_ds = ComVEDataset.gpt2_lm_plain(data_folder='/content/gdrive/MyDrive/git/data',
                                                               training_x='GPT2_data_final_prod.csv',
                                                               training_y='GPT2_answers_final_prod.csv',
                                                               x_transforms=[tokenizer], 
                                                               aug_size=0)
        train_dataloader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
        val_dataloader = DataLoader(val_ds, batch_size=len(val_ds))
        test_dataloader = DataLoader(test_ds, batch_size=len(test_ds))
        training_steps = len(train_dataloader) * EPOCHS
        model = TransformerBasedClassifier(name=name, 
                                           drop=DROP, 
                                           lr=4e-5, 
                                           weight_decay=0.01, 
                                           training_steps=training_steps)
        checkpoint_callback = ModelCheckpoint(
            monitor = 'val_loss',
            dirpath = 'checkpoint/',
            filename = 'best-model-' + name,
            save_top_k = 1
        )
        trainer = pl.Trainer(max_epochs=EPOCHS, 
                             gpus=1, 
                             progress_bar_refresh_rate=20, 
                             callbacks=[])
        trainer.fit(model, train_dataloader, val_dataloader)
        l, a = model.get_best_loss_and_acc()
        model = TransformerBasedClassifier.load_from_checkpoint(checkpoint_callback.best_model_path,
                                                                name=name,
                                                                drop=DROP,
                                                                append_name=f'aug_size={size}')
        model.val_loss, model.val_acc = l, a
        trainer.test(model, test_dataloader)
    !rm -rf checkpoint/